In [1]:
!pip install transformers
!pip install seqeval
!pip install sentencepiece

     |████████████████████████████████| 1.5MB 8.2MB/s 
     |████████████████████████████████| 2.9MB 27.5MB/s 
     |████████████████████████████████| 890kB 49.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1307369f70fa5e33d7ef99e34dc537ea4c0ed585d4f7903bd5c3a0ced3ded22e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 4.8MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=1c3679f9d194d1d82bce4a2ec86a22688750e2182d77ec821f33ccdc5d586bbb
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
     |████████████████████████████████| 1.1MB 7.2MB/s 


In [2]:
!wget https://github.com/wannaphong/thai-ner/raw/master/model/1.3/train.data

--2020-12-30 21:36:08--  https://github.com/wannaphong/thai-ner/raw/master/model/1.3/train.data
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wannaphong/thai-ner/master/model/1.3/train.data [following]
--2020-12-30 21:36:08--  https://raw.githubusercontent.com/wannaphong/thai-ner/master/model/1.3/train.data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11598502 (11M) [application/octet-stream]
Saving to: ‘train.data’

train.data          100%[===================>]  11.06M  48.8MB/s    in 0.2s    

2020-12-30 21:36:09 (48.8 MB/s) - ‘train.data’ saved [11598502/11598502]



In [3]:
import dill
with open('train.data', 'rb') as file:
 datatofile = dill.load(file)

In [4]:
import torch
from transformers import BertTokenizer, BertForTokenClassification, AdamW, XLMRobertaTokenizer,XLMRobertaForTokenClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from seqeval.scheme import IOBES
import itertools
import sentencepiece

import numpy as np

In [5]:
#tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


In [6]:
sentences = [[word[0] for word in sentence] for sentence in datatofile]
true_labels = [[s[2] for s in sentence] for sentence in datatofile]

In [7]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [8]:
tag_values = list(set(list(itertools.chain(*true_labels))))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [9]:
X_train, X_test, y_train, y_test = train_test_split(sentences, true_labels, test_size=0.2, shuffle=False)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [11]:
MAX_LEN = 200
bs = 32

In [12]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []
    word_index = []

    for i,(word, label) in enumerate(zip(sentence, text_labels)):
      if word == ' ':
        word = '_'

      # Tokenize the word and count # of subwords the word is broken into
      tokenized_word = tokenizer.tokenize(word)
      n_subwords = len(tokenized_word)

      # Add the tokenized word to the final tokenized word list
      tokenized_sentence.extend(tokenized_word)

      # Add the same label to the new list of labels `n_subwords` times
      labels.extend([label]*(n_subwords))
      

      #Add word index
      word_index.extend([i]*n_subwords)

    return tokenized_sentence, labels, word_index

In [13]:
tokenized_texts_and_labels_train = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(X_train, y_train)
]


tokenized_texts_and_labels_test = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(X_test, y_test)
]


In [14]:
tokenized_texts_train = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_train]
labels_train = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_train]
word_index_train = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels_train]

tokenized_texts_test = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_test]
labels_test = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_test]
word_index_test = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels_test]

In [15]:
def combine_word(token_sentence, word_index, token_labels):
  last_index = 0
  temp_word = []
  temp_sentence = []
  labels = [token_labels[0]]
  for i, index in enumerate(word_index):
    if index != last_index:
      word = ''.join(temp_word).replace('##','').replace('_', ' ')
      temp_sentence.append(word)
      temp_word = [token_sentence[i]]
      last_index = index
      labels.append(token_labels[i])
    else:
      subword = token_sentence[i]
      temp_word.append(subword)
      last_index = index
  word = ''.join(temp_word).replace('##','')
  temp_sentence.append(word)
  labels.append(token_labels[i])

  return temp_sentence,labels

In [16]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_train],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [17]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_train],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [18]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [19]:
input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [20]:
tags_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_test],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [21]:
attention_masks_test = [[float(i != 0.0) for i in ii] for ii in input_ids_test]

In [22]:
tr_inputs = torch.tensor(input_ids)
tr_tags = torch.tensor(tags)
tr_masks = torch.tensor(attention_masks)

val_inputs = torch.tensor(input_ids_test)
val_tags = torch.tensor(tags_test)
val_masks = torch.tensor(attention_masks_test)

In [23]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [24]:
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', 
                                                   num_labels = len(tag2idx),
                                                   output_attentions = False,
                                                   output_hidden_states = False
                                                   )

#model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base',
#                                         num_labels = len(tag2idx),
#                                         output_attentions = False,
#                                         output_hidden_states = False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [25]:
model.cuda();

In [26]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [27]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [28]:
from seqeval.metrics import f1_score, accuracy_score,classification_report
from tqdm import tqdm, trange

In [29]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score([pred_tags], [valid_tags])))
    print()

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Average train loss: 0.5104700280060558
Validation loss: 0.31367518827319146
Validation Accuracy: 0.9183767868420819


Epoch:  33%|███▎      | 1/3 [03:21<06:43, 201.65s/it]

Validation F1-Score: 0.6786617225686836

Average train loss: 0.19521618095583887
Validation loss: 0.22958705853670835
Validation Accuracy: 0.9382399772280486


Epoch:  67%|██████▋   | 2/3 [06:47<03:22, 202.91s/it]

Validation F1-Score: 0.7767047363117232

Average train loss: 0.1296561251646318
Validation loss: 0.21205154955387115
Validation Accuracy: 0.9440041274161841


Epoch: 100%|██████████| 3/3 [10:13<00:00, 204.58s/it]

Validation F1-Score: 0.7999495034505976



In [30]:
def get_label(true_val, pred_val, attention_masks, word_index):
  true_with_masking = true_val.cpu().numpy()*attention_masks
  output_with_masking = pred_val.cpu().numpy()*attention_masks

  true_with_masking = [i for i in true_with_masking if i != 0]
  output_with_masking = [i for i in output_with_masking if i != 0]

  _pred = [output_with_masking[0]]
  _true = [true_with_masking[0]]
  last_index = 0

  for i,index in  enumerate(word_index[:200]):
    if index != last_index:
      _pred.append(output_with_masking[i])
      _true.append(true_with_masking[i])
      last_index = index 
    
  pred = [tag_values[int(value)] for value in _pred]
  true = [tag_values[int(value)] for value in _true]

  return true, pred

In [39]:
all_true = []
all_pred = []
error = []
for i, input_id_test in enumerate(input_ids_test):
  outputs = model(torch.tensor([input_ids_test[i]]).to(device))
  output_logits = torch.argmax(outputs['logits'],axis=2)[0]
  try:
    true, pred = get_label(val_tags[i], output_logits, attention_masks_test[i], word_index_test[i])
  except:
    error.append(i)
  all_true.append(true)
  all_pred.append(pred)

In [40]:
f1_score(all_true, all_pred)

0.6764355086045323

In [41]:
print(classification_report(all_true, all_pred))

/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        DATE       0.88      0.79      0.84       312
         LAW       0.12      0.08      0.10        73
         LEN       0.64      0.64      0.64        11
    LOCATION       0.76      0.65      0.70       578
       MONEY       0.71      0.53      0.61       122
ORGANIZATION       0.67      0.68      0.67      1221
     PERCENT       0.14      0.21      0.17        14
      PERSON       0.65      0.71      0.68       563
       PHONE       0.00      0.00      0.00         2
        TIME       0.64      0.56      0.60        80
         URL       0.67      0.67      0.67         9
   ไม่ยืนยัน       0.00      0.00      0.00         1

   micro avg       0.69      0.66      0.68      2986
   macro avg       0.49      0.46      0.47      2986
weighted avg       0.69      0.66      0.68      2986

